In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
from dask import dataframe as dd
pd.options.mode.chained_assignment = None

ensemble = True

In [2]:
transactions = dd.read_csv('data/transactions_train.csv', dtype={'article_id': str})
if ensemble:
    transactions = transactions[(transactions.t_dat >= '2020-03-17') & (transactions.t_dat < '2020-09-15')]
else:
    transactions = transactions[transactions.t_dat >= '2020-03-24']
transactions.t_dat = dd.to_datetime(transactions.t_dat) - timedelta(2)
transactions["week"] = transactions.t_dat.dt.isocalendar().week
transactions = transactions.compute()
transactions["rebuy_count"] = transactions.groupby(["customer_id", "article_id"]).cumcount().astype(int)
transactions["rebuy_count"] = transactions.rebuy_count.apply(lambda x: x -1 if x > 0 else 0)

In [3]:
most_solds = transactions[transactions.week == transactions.week.max()].groupby(["article_id"]).agg({"article_id":"count"})\
                                   .rename(columns={"article_id":"_count"}).reset_index()\
                                   .sort_values('_count', ascending=False)
most_solds = most_solds.head(10000)
transactions = transactions[transactions.article_id.isin(most_solds.article_id)]
articles = pd.read_csv("data/articles.csv", dtype={'article_id': str})
articles = articles[articles.article_id.isin(most_solds.article_id)]
del most_solds

In [4]:
active_customers = transactions.customer_id.unique()
customers = pd.read_csv("data/customers.csv", dtype={'customer_id': str})
customers = customers[customers.customer_id.isin(active_customers)]
del active_customers

In [5]:
def gender_classification(age):
    if age < 19:
        return 0
    elif age < 29:
        return 1
    elif age < 49:
        return 2
    elif age < 59:
        return 3
    elif age < 69:
        return 4
    else:
        return 5

customers["age"] = customers.age.fillna(np.mean(customers.age))
customers["age_bin"] = customers.age.map(gender_classification)

In [6]:
def gender_classification(section_name):
    if "womens" in section_name or "girl" in section_name or "ladies" in section_name:
        return "woman"
    elif "men" in section_name or "boy" in section_name or "boys" in section_name:
        return "man"
    else:
        return "other"

articles.section_name = articles.section_name.map(lambda x: x.lower())
articles["gender_group"] = articles.section_name.apply(gender_classification)

In [7]:
transactions = transactions.merge(articles[["article_id", "gender_group"]], on="article_id", how="inner")
transactions = transactions.merge(customers[["customer_id","age_bin"]], on="customer_id", how="inner")
customer_hist = transactions.groupby(by="customer_id").agg({"article_id": lambda x: list(x.values), "week": lambda x: list(x.values), "gender_group": lambda x : x.mode().iloc[0]}).reset_index()
customers = customers.merge(customer_hist, on="customer_id", how="left")
customers.article_id = customers.article_id.fillna("").apply(list)
customers.gender_group = customers.gender_group.fillna("other")
transactions.price = transactions.price.fillna(transactions.price.mean())
prod_price = transactions.groupby("customer_id").agg({"price":"mean"}).rename(columns={"customer_id":"price_"}).reset_index()
prod_price.columns = list(map(''.join, prod_price.columns.values))
customers = customers.merge(prod_price, on="customer_id", how="inner")
del customer_hist

In [8]:
customers["history"] = customers.apply(lambda x: sorted(zip(x.week, x.article_id)), axis=1)
customers.FN = customers.FN.fillna(0)
customers.Active = customers.Active.fillna(0)
customers.fashion_news_frequency = customers.fashion_news_frequency.fillna("not_regular")
customers.club_member_status = customers.club_member_status.fillna("no-info")
customers.fashion_news_frequency = customers.fashion_news_frequency.apply(lambda x: "not_regular" if x == "NONE" or x == "None" else x)
customers["numberOfArticles"] = customers.apply(lambda x: len(x.history), axis=1)
customers = customers.drop(columns=["postal_code", "week", "article_id"])

In [9]:
def map_fashion_news(name):
    return 1 if name == 'not_regular' else 0
def map_club_member(name):
    return 1 if name == 'ACTIVE' else 0

customers.fashion_news_frequency = customers.fashion_news_frequency.map(map_fashion_news)
customers.club_member_status = customers.club_member_status.map(map_club_member)

In [10]:
def article2doc(x):
    def clean_doc(text):
        unwanted_chars = ['1','2','3','4','5','6','7','8','9','(',')','[',']']
        for chr in unwanted_chars:
            text = text.replace(chr, '')
        return text

    doc =  '. '.join([x.prod_name, x.product_type_name, x.product_group_name, x.graphical_appearance_name, x.colour_group_name,\
                      x.perceived_colour_value_name, x.perceived_colour_master_name, x.department_name, x.index_name, x.index_group_name,\
                      x.section_name, x.garment_group_name, str(x.detail_desc)])[:-1]
    return(clean_doc(doc.lower()))

articles["doc"] = articles.apply(article2doc, axis=1)
article_info = transactions.groupby(["article_id"]).agg({"price":"mean", "rebuy_count":"sum", "age_bin": lambda x : x.mode().iloc[0]}).rename(columns={"price":"prod_avg_price","rebuy_count":"prod_rebuy_count","age_bin":"prod_age_bin"}).reset_index()
articles = articles.merge(article_info, on="article_id", how="inner")
most_solds = transactions[transactions.week > transactions.week.max()-3].groupby(["article_id"]).agg({"customer_id":"count"})\
                                      .rename(columns={"customer_id":"prod_sold_count"}).reset_index()
articles = articles.merge(most_solds, on="article_id", how="inner")
articles = articles[["article_id","doc","gender_group", "prod_avg_price", "prod_rebuy_count", "prod_age_bin", "prod_sold_count"]]
weekly_sales = transactions.groupby(["article_id", "week"]).agg({"customer_id":"count"}).reset_index()
last_week_sales = weekly_sales[weekly_sales.week == weekly_sales.week.max()]
weekly_sales = weekly_sales.merge(last_week_sales[["article_id","customer_id"]], on=["article_id"], how="inner")
weekly_sales["quotient"] = weekly_sales.customer_id_y / weekly_sales.customer_id_x
articles = articles.merge(weekly_sales[["article_id","quotient"]], on="article_id", how="inner")

del article_info, most_solds, weekly_sales, last_week_sales

In [11]:
customers["doc"] = customers.history.apply(lambda x: list(set([i[1] for i in x])))
temp_dict = {}
for i,row in articles.iterrows():
    temp_dict[row.article_id] = row.doc

customers["doc"] = customers.doc.apply(lambda x:  ". ".join([temp_dict[i] for i in x]))

In [12]:
customers.head(3)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,age_bin,gender_group,price,history,numberOfArticles,doc
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.0,0.0,1,1,49.0,3,woman,0.032610,"[(12, 0795440001), (36, 0568601043)]",2,juan. vest top. garment upper body. solid. bla...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.0,0.0,1,1,25.0,1,woman,0.022017,"[(17, 0351484002), (17, 0559630026), (17, 0599...",9,timeless midrise brief. swimwear bottom. swimw...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0,0.0,1,1,24.0,1,man,0.025407,"[(16, 0852643001), (16, 0852643003)]",2,pez sweater. sweater. garment upper body. soli...


In [13]:
articles.head()

,article_id,doc,gender_group,prod_avg_price,prod_rebuy_count,prod_age_bin,prod_sold_count,quotient
0,0108775044,strap top. vest top. garment upper body. solid...,woman,0.007931,27,1,15,3.000000
1,0108775044,strap top. vest top. garment upper body. solid...,woman,0.007931,27,1,15,0.750000
2,0108775044,strap top. vest top. garment upper body. solid...,woman,0.007931,27,1,15,1.500000
3,0108775044,strap top. vest top. garment upper body. solid...,woman,0.007931,27,1,15,0.333333
4,0108775044,strap top. vest top. garment upper body. solid...,woman,0.007931,27,1,15,0.142857


## Content Based Corpus Data Creating

In [14]:
doc_df1 = articles[["article_id","doc"]].rename(columns={"article_id":"doc_id"}).copy()
articles.drop(columns=["doc"], inplace=True)
doc_df2 = customers[["customer_id","doc"]].rename(columns={"customer_id":"doc_id"}).copy()
customers.drop(columns=["doc"], inplace=True)
doc_df1["type"] = "product"
doc_df2["type"] = "customer"
doc_df = doc_df1.append(doc_df2)
doc_df.to_csv("data/corpus.csv", index=False)

del doc_df, doc_df1, doc_df2

## Collabrative Data Creating

In [15]:
transactions[["customer_id","article_id","week"]].to_pickle("data/transactions.pkl")
last_week = transactions.week.max()
del transactions

## Ensemble Data Creating

In [16]:
customers.FN = customers.FN.astype('category').cat.codes
customers.Active = customers.Active.astype('category').cat.codes
customers.club_member_status = customers.club_member_status.astype('category').cat.codes
customers.fashion_news_frequency = customers.fashion_news_frequency.astype('category').cat.codes
customers.age_bin = customers.age_bin.astype('category').cat.codes
customers.gender_group = customers.gender_group.astype('category').cat.codes
customers.age = (customers.age - customers.age.min()) / (customers.age.max() - customers.age.min())
customers.numberOfArticles = (customers.numberOfArticles - customers.numberOfArticles.min()) / (customers.numberOfArticles.max() - customers.numberOfArticles.min())

In [17]:
articles.gender_group = articles.gender_group.astype('category').cat.codes
articles.prod_age_bin = articles.prod_age_bin.astype('category').cat.codes
articles.prod_avg_price = (articles.prod_avg_price - articles.prod_avg_price.min()) / (articles.prod_avg_price.max() - articles.prod_avg_price.min())
articles.prod_rebuy_count = (articles.prod_rebuy_count - articles.prod_rebuy_count.min()) / (articles.prod_rebuy_count.max() - articles.prod_rebuy_count.min())
articles.prod_sold_count = (articles.prod_sold_count - articles.prod_sold_count.min()) / (articles.prod_sold_count.max() - articles.prod_sold_count.min())
articles.quotient = (articles.quotient - articles.quotient.min()) / (articles.quotient.max() - articles.quotient.min())

In [92]:
data = dd.read_csv('data/transactions_train.csv', dtype={'article_id': str})[["customer_id", "article_id","t_dat"]]
data["week"] = last_week + 1
data = data.drop_duplicates(subset=["customer_id", "article_id"])
data = data[data.t_dat > '2020-09-15'].compute()

In [93]:
data.t_dat.unique()

array(['2020-09-16', '2020-09-17', '2020-09-18', '2020-09-19',
       '2020-09-20', '2020-09-21', '2020-09-22'], dtype=object)

In [50]:
data = data.merge(articles, on="article_id", how="inner")
data = data.merge(customers, on="customer_id", how="inner")
#data["customer_rebuy_count"] = data.apply(lambda x: [i[1] for i in x.history].count(x.article_id), axis=1)

In [86]:
data["prod_hist"] = data.history.apply(lambda x: [i[1] for i in x])

In [88]:
data["customer_rebuy_count"] = data.apply(lambda x: x.prod_hist.count(str(x.article_id)), axis=1)

In [91]:
data.

(1575563, 21)

In [90]:
data[data.customer_rebuy_count > 0]

,customer_id,article_id,week,gender_group_x,prod_avg_price,prod_rebuy_count,prod_age_bin,prod_sold_count,quotient,FN,...,club_member_status,fashion_news_frequency,age,age_bin,gender_group_y,price,history,numberOfArticles,customer_rebuy_count,prod_hist


In [68]:
data["customer_rebuy_count"] = data.apply(lambda x: [i[1] for i in x.history].count(x.article_id), axis=1)

In [82]:
data[data.customer_rebuy_count > 0]

,customer_id,article_id,week,gender_group_x,prod_avg_price,prod_rebuy_count,prod_age_bin,prod_sold_count,quotient,FN,Active,club_member_status,fashion_news_frequency,age,age_bin,gender_group_y,price,history,numberOfArticles,customer_rebuy_count


In [77]:
c_hist = [i[1] for i in data[data.customer_id == "000fb6e772c5d0023892065e659963da90b1866035558ec16fca51b0dcfb7e59"].head().history[0]]

In [81]:
'0871519008' in c_hist

False

In [25]:
def get_last_purchased_time(x):
    week_passed = 7*4
    if x.customer_rebuy_count != 0:
        for i in x.history:
            if i[1] == x.article_id:
                week_passed = i[0]
    return week_passed

data["customer_last_time_same_prod"] = data.apply(lambda x: get_last_purchased_time(x), axis=1)
data.drop(columns=["history"], inplace=True)

In [73]:
data.head().customer_id[0]

'000fb6e772c5d0023892065e659963da90b1866035558ec16fca51b0dcfb7e59'

In [33]:
x = (data.customer_rebuy_count - data.customer_rebuy_count.min()) / (data.customer_rebuy_count.max() - data.customer_rebuy_count.min())

In [43]:
data[data.notna()]

,customer_id,article_id,week,gender_group_x,prod_avg_price,prod_rebuy_count,prod_age_bin,prod_sold_count,quotient,FN,Active,club_member_status,fashion_news_frequency,age,age_bin,gender_group_y,price,numberOfArticles,customer_rebuy_count,customer_last_rebuy_week
0,000fb6e772c5d0023892065e659963da90b1866035558e...,0786022008,38,2,0.118082,0.000000,1,0.014171,0.001983,1,1,1,0,0.313253,2,2,0.032459,0.184275,0,28
1,000fb6e772c5d0023892065e659963da90b1866035558e...,0913272003,38,2,0.087138,0.000645,2,0.097780,0.053673,1,1,1,0,0.313253,2,2,0.032459,0.184275,0,28
2,000fb6e772c5d0023892065e659963da90b1866035558e...,0913272003,38,2,0.087138,0.000645,2,0.097780,0.001755,1,1,1,0,0.313253,2,2,0.032459,0.184275,0,28
3,000fb6e772c5d0023892065e659963da90b1866035558e...,0913272003,38,2,0.087138,0.000645,2,0.097780,0.001137,1,1,1,0,0.313253,2,2,0.032459,0.184275,0,28
4,000fb6e772c5d0023892065e659963da90b1866035558e...,0913272003,38,2,0.087138,0.000645,2,0.097780,0.001983,1,1,1,0,0.313253,2,2,0.032459,0.184275,0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575558,d17b9da096c06932220966c42bd0dbc9f46856588acef2...,0784384003,38,2,0.087259,0.000860,2,0.008030,0.002480,1,1,1,0,0.385542,2,2,0.027478,0.019656,0,28
1575559,d17b9da096c06932220966c42bd0dbc9f46856588acef2...,0784384003,38,2,0.087259,0.000860,2,0.008030,0.003308,1,1,1,0,0.385542,2,2,0.027478,0.019656,0,28
1575560,d17b9da096c06932220966c42bd0dbc9f46856588acef2...,0784384003,38,2,0.087259,0.000860,2,0.008030,0.002480,1,1,1,0,0.385542,2,2,0.027478,0.019656,0,28
1575561,d17b9da096c06932220966c42bd0dbc9f46856588acef2...,0784384003,38,2,0.087259,0.000860,2,0.008030,0.000989,1,1,1,0,0.385542,2,2,0.027478,0.019656,0,28


In [44]:
data.shape

(1575563, 20)

In [46]:
(data.customer_rebuy_count - data.customer_rebuy_count.min()) / 2

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
1575558    0.0
1575559    0.0
1575560    0.0
1575561    0.0
1575562    0.0
Name: customer_rebuy_count, Length: 1575563, dtype: float64

In [51]:
data.customer_last_rebuy_week.max()

AttributeError: 'DataFrame' object has no attribute 'customer_last_rebuy_week'

In [ ]:
data.customer_rebuy_count = (data.customer_rebuy_count - data.customer_rebuy_count.min()) / (data.customer_rebuy_count.max() - data.customer_rebuy_count.min())
data.customer_last_rebuy_week = 1 - ((data.customer_last_rebuy_week - data.customer_last_rebuy_week.min()) / (data.customer_last_rebuy_week.max() - data.customer_last_rebuy_week.min()))